# Mise en forme pour un graphe à barres de largeur variable

#### Importation des modules

- codecs : gestion des fichiers utf8 (open)
- sys : gestion du système (exit)

In [19]:
import codecs,sys

#### Définition du séparateur et du nom de fichier

In [20]:
sep="\t"
File="/Users/gilles/Downloads/MotsDureeSpeaker.csv"
console=False

####Ouverture du fichier

In [21]:
try:
    fileContent=codecs.open(File,"r", "utf-8") 

except IOError:
    print 'I could not open the file', File
    sys.exit()

if not console:
    Output=File.split(".")[0]+"-Var.csv"
    try:
        fileOutput=codecs.open(Output,"w", "utf-8") 

    except IOError:
        print 'I could not open the file', Output
        sys.exit()


#### Calcul de l'indice de temps en centièmes de secondes

Les champs de temps sont prévus sous la forme "2014-07-14 00:12:24.600000" pour 12 minutes 24 secondes et 60 centièmes

- calcul des centièmes
- différence avec le temps initial

In [22]:
base=0
def formatTime(date):
    jour,instant=date.split(" ")
    heures, minutes, secmil = instant.split(":")
    if "." in secmil:
        secondes, milliemes=secmil.split(".")
    else:
        secondes=secmil
        milliemes="0"
    numero=int(milliemes)/10000+100*int(secondes)+6000*int(minutes)+360000*int(heures)
    return numero-base

####Transformation d'une ligne de données

- découpage de *line* sur le séparateur dans *champs*
- transformation des deux premiers *champs* en nombre de centièmes de seconde depuis le premier top (*base*)
- accumulation des champs restants dans *valeurs*
 - nettoyage des chaines *valeur*
 - remplacement des chaines vides par des "0"
 - accumulation de "0" dans *zeros*, autant que de *valeur* dans *valeurs*, pour faire les bas de barres

La fonction renvoie deux chaines (*debut*, *fin*) et deux listes (*valeurs*, *zeros*)

In [23]:
def formatLine(line):
    champs=line.split(sep)
    debut=str(formatTime(champs[1]))
    fin=str(formatTime(champs[2]))
    valeurs=[]
    zeros=[]
    for champ in champs[3:]:
        valeur=champ.strip()
        if valeur=="" :
            valeurs.append("0")
        else:
            valeurs.append(valeur)
        zeros.append("0")
    valeurs.extend(["",champs[1]])
    zeros.extend(["",champs[1]])
    return (debut, fin, valeurs, zeros)

### Transformation des lignes pour les barres à largeur-durée

On lit le fichier et on découpe la première ligne pour mettre en place les titres

- on élimine le numéro d'index *titres[0]* et l'étiquette fin *titres[2]*, on garde les autres titres
- l'instant de référence est le debut sur la *lignes[1]* => *base* 

- au départ on a une ligne avec un numéro, le début, la fin et les autres valeurs
- à l'arrivée on a quatre lignes
 - le début avec des zéros
 - le début avec les autres valeurs
 - la fin avec les autres valeurs
 - la fin avec des zéros

In [24]:
lignes=fileContent.readlines()
titres=lignes[0].split("\t")
champs=formatLine(lignes[1])
base=int(champs[0])

if console:
    print titres[1].encode("utf8")+sep+sep.join(titres[3:]).strip().encode("utf8")
    for ligne in lignes[1:50]:
        (debut,fin,valeurs,zeros)=formatLine(ligne)
        if valeurs!=zeros:
            print debut+sep+sep.join(zeros)
            print debut+sep+sep.join(valeurs)
            print fin+sep+sep.join(valeurs)
            print fin+sep+sep.join(zeros)
else:
    fileOutput.write(titres[1]+sep+sep.join(titres[3:]).strip()+"\n")
    for ligne in lignes[1:]:
        (debut,fin,valeurs,zeros)=formatLine(ligne)
        if valeurs!=zeros:
            fileOutput.write(debut+sep+sep.join(zeros)+"\n")
            fileOutput.write(debut+sep+sep.join(valeurs)+"\n")
            fileOutput.write(fin+sep+sep.join(valeurs)+"\n")
            fileOutput.write(fin+sep+sep.join(zeros)+"\n")
    print Output, "Terminé"
    

/Users/gilles/Downloads/MotsDureeSpeaker-Var.csv Terminé
